# LLM Calls

In [4]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
# llm = OpenAI(model="gpt-3.5-turbo")
chat_model = ChatOpenAI(model="gpt-3.5-turbo")
# llm.predict("hi!")
chat_model.predict("Hi")

'Hello! How can I assist you today?'

# Download competitions

In [15]:
from kaggle.api.kaggle_api_extended import KaggleApi
api = KaggleApi()
api.authenticate()

In [17]:
# api.dataset_download_file('kaggle/meta-kaggle/download?datasetVersionNumber=1131',
#                           file_name='Competitions.csv',
#                           path='./data')

api.dataset_download_file('kaggle/meta-kaggle/download?datasetVersionNumber=1131',
                          file_name='ForumTopics.csv',
                          path='./data')

True

In [21]:
import zipfile

def extract_zip(name):
    zip_path = f'./data/{name}.csv.zip'  
    extract_path = './data'  

    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)

extract_zip("ForumTopics")

In [2]:
import pandas as pd
forums_df = pd.read_csv("./data/ForumTopics.csv")
forums_df.head()

,Id,ForumId,KernelId,LastForumMessageId,FirstForumMessageId,CreationDate,LastCommentDate,Title,IsSticky,TotalViews,Score,TotalMessages,TotalReplies
0,46,1,NaN,684511.0,220.0,08/03/2010 22:24:40,11/30/2019 00:07:53,The solution,False,11409,0,8,7
1,6589,1,NaN,554419.0,36135.0,12/12/2013 22:18:34,06/17/2019 11:42:33,What is H in test-data?,False,1940,0,3,2
2,36,1,NaN,141.0,134.0,06/30/2010 20:20:10,06/30/2010 20:20:10,Scoring system,False,4781,0,4,3
3,119608,1,NaN,1667283.0,684518.0,11/30/2019 00:23:31,01/28/2022 12:52:13,Submissions Closed/Solution Data Unavailable,False,0,0,5,4
4,62,1,NaN,402.0,328.0,08/11/2010 18:12:05,08/11/2010 18:12:05,Accuracy on full data set,False,6241,0,3,2


In [3]:
comp_df = pd.read_csv("./data/Competitions.csv")
# we can filter to keep only the best competitions, i.e. "Featured", "Reseach" and recruitments

In [17]:

comp_slug = "Eurovision2010"
comp_url = "https://www.kaggle.com/c/"+str(comp_slug)
comp_url

'https://www.kaggle.com/c/Eurovision2010'

# Scrapping a competition

In [10]:
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


options = webdriver.ChromeOptions()
options.add_argument("ignore-certificate-errors")
# options.add_argument("--headless")
driver = webdriver.Chrome(options=options) 

def get_page_content(url):
    driver.get(url)

    element = WebDriverWait(driver, 10).until(
        # EC.presence_of_element_located((By.ID, "pageheader-nav-item--overview"))
        EC.presence_of_element_located((By.ID, "background"))
    )
    soup = BeautifulSoup(driver.page_source, 'html.parser').find("div", {"id": "background"}).find_parent("div")
    return soup, soup.get_text()

soup, content = get_page_content('http://www.kaggle.com/c/Eurovision2010')
soup

<div class="sc-fOVJBw gEnjkX"><div class="sc-jhJPuH lexbYk" id="background"><div class="sc-dskThN cGppFq sc-ealCdV jZWWSy"><div class="sc-cYRmzm SKQgu"><div class="sc-jVIkER hIVbFL" id="Background"><div class="sc-iHpopC bLEUds"><h5 class="sc-brKeYL sc-ktEKTO izsmej JpJng">Background</h5></div><button aria-label="Copy link" class="sc-ejdXBC jEohRd undefined google-material-icons" title="Copy link">link</button></div><button aria-label="Collapse " class="sc-ejdXBC jEohRd undefined google-material-icons" style="flex-shrink: 0;" title="Collapse ">keyboard_arrow_up</button></div><div><div class="sc-SQxqA hAiBil"><div class="sc-dlyeST hCUClb">The Eurovision Song Contest is an annual competition broadcast worldwide which is open to entrants from active members of the European Broadcasting Union (EBU).  This is not defined by Europe's geographical bounds; for example Israel and Cyprus compete in the contest.  This year's contest is being held in Oslo, Norway.  The location is usually the count

In [11]:
content

'Backgroundlinkkeyboard_arrow_upThe Eurovision Song Contest is an annual competition broadcast worldwide which is open to entrants from active members of the European Broadcasting Union (EBU).\xa0 This is not defined by Europe\'s geographical bounds; for example Israel and Cyprus compete in the contest.\xa0 This year\'s contest is being held in Oslo, Norway.\xa0 The location is usually the country of the previous year\'s winner.\xa0 The number of entrants changes on a yearly basis, due to flux in EBU membership, political shifts such as the dissolution of the Former Soviet Union and Yugoslavia, and desire (or lack thereof) of countries to participate.\xa0 \xa0Each country must submit one song to represent it, which must not have been previously commercially released.\xa0 All songs are, however, made public by a deadline several weeks before the contest, and this year\'s entrants are all known already with details such as title, language and artist included in the database provided.\xa0

In [6]:
soup.get_text()

'Citationkeyboard_arrow_upAnthony Goldbloom. (2010). Forecast Eurovision Voting . Kaggle. https://kaggle.com/competitions/Eurovision2010Cite'

# Format competition summary and problem

In [27]:
from langchain.prompts import PromptTemplate



def get_formated_comp_description(text):
    prompt = PromptTemplate.from_template("""Here is data scrapped from kaggle competition. Please format your response this way:
    1. Explain the problem for a machine learning expert
    2. Give the category of the problem
    {competition_text}""")
    return chat_model.predict(prompt.format(competition_text=text))

# Get solutions URL for a competition

In [44]:
def get_discussion_results(forum_id, n):
    results_df = forums_df[forums_df["ForumId"]==forum_id]
    results_df = results_df.sort_values(by=["Score","TotalMessages"], ascending=False).head(n).reset_index(drop=True)
    return results_df[["Title", "Id", "Score", "TotalMessages", "TotalReplies"]]

In [51]:
def get_discussions_urls(comp_id):
    id_forum_comp = comp_df.iloc[comp_id]["ForumId"]
    comp_slug = comp_df.iloc[comp_id]["Slug"]

    discussions_df = get_discussion_results(id_forum_comp, 10)
    url_list = []
    for id_disc, title, score in zip(discussions_df["Id"], discussions_df["Title"], discussions_df["Score"]):
        url = f"https://www.kaggle.com/c/{comp_slug}/discussion/{id_disc}"
        url_list.append((title, url, score))
    
    return url_list

candidate_solutions = get_discussions_urls(5603) 
candidate_solutions


[('Solved, without any submission. This Crab has Eleven Years Kaggling.',
  'https://www.kaggle.com/c/playground-series-s3e16/discussion/413750',
  62),
 ('3rd private 6th public | Brute-force ensemble| Post-processing',
  'https://www.kaggle.com/c/playground-series-s3e16/discussion/416783',
  62),
 ('Feature engineering that helped me reduce MAE to 1.33708',
  'https://www.kaggle.com/c/playground-series-s3e16/discussion/415721',
  49),
 ('Some Domain Materials Regarding the Dataset',
  'https://www.kaggle.com/c/playground-series-s3e16/discussion/413741',
  45),
 ('To round or not to round?',
  'https://www.kaggle.com/c/playground-series-s3e16/discussion/413971',
  35),
 ('Tips and Tricks to help you improve your score. ( Part 2)',
  'https://www.kaggle.com/c/playground-series-s3e16/discussion/415200',
  27),
 ('#1 Solution - Experimenting with the data generation model',
  'https://www.kaggle.com/c/playground-series-s3e16/discussion/416766',
  23),
 ("The Most Bizarre Feature Correlat

# Scrapping of URL solution
For a given URL solution, we collect its explanation 

In [53]:
candidate_solutions[3][1]

'https://www.kaggle.com/c/playground-series-s3e16/discussion/413741'

In [86]:
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


def get_solution_scrapping(url):
    options = webdriver.ChromeOptions()
    options.add_argument("ignore-certificate-errors")
    options.add_argument("--headless")
    driver = webdriver.Chrome(options=options) 

    driver.get(url)

    WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.CLASS_NAME, "sc-bZMpAI"))
    )

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    return soup.find(class_="sc-dlyeST")
get_solution_scrapping(candidate_solutions[9][1])

<div class="sc-dlyeST hCUClb"><p>Hello everyone, first of all, I'd like to mention that this is my first Kaggle competition at the global level. Although I consider myself a novice in this field, I have been actively involved in Data Science/Machine Learning Engineering for about a year, and I knew I was good at it. So, being the second in my first competition made me really happy.</p>
<p>Soon, I will share my approach and notebook on this Discord channel : <a href="https://discord.gg/WpGhjsz8pn" rel="noreferrer nofollow" target="_blank">https://discord.gg/WpGhjsz8pn</a></p>
<p>For now, let me briefly explain my approach:</p>
<p>1) First and foremost, I heavily relied on using the class structure and conducted numerous trial and error experiments. I tried almost everything, such as testing different encoders to see which one provides better results, deciding which features to add, and determining which approaches yield better outcomes. I tried various things by adopting an open-close a

# Storing and retrieval of embeddings 

In [20]:
soup, content = get_page_content('http://www.kaggle.com/c/Eurovision2010')

In [28]:
formated_comp_desc = get_formated_comp_description(content)

In [29]:
formated_comp_desc

'1. The problem for a machine learning expert is to predict the voting outcomes for the 2010 Eurovision Song Contest. This involves predicting which 25 countries will make it to the final and then predicting the voting between these countries. The goal is to create a model that accurately predicts the voting results based on historical voting patterns, betting odds, song details, and artist details.\n\n2. The category of the problem is a classification and prediction problem. It involves predicting the categorical outcome of which countries will make it to the final and then predicting the numerical outcome of the voting scores between these countries.'

In [1]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.docstore.document import Document

def create_document(content):
    return Document(page_content=content, metadata={"source": "local"})

# embeddings = OpenAIEmbeddings.embed_documents(texts=[content])
documents = [create_document(formated_comp_desc), create_document("This is a competition where we have a time serie forecasting problem"), create_document("this is a random string")]

vectorstore = Chroma.from_documents(documents=documents, embedding=OpenAIEmbeddings())

NameError: name 'formated_comp_desc' is not defined

In [33]:
question = "Weather prediction"
docs = vectorstore.similarity_search(question)
docs

[Document(page_content='This is a competition where we have a time serie forecasting problem', metadata={'source': 'local'}),
 Document(page_content='This is a competition where we have a time serie forecasting problem', metadata={'source': 'local'}),
 Document(page_content='1. The problem for a machine learning expert is to predict the voting outcomes for the 2010 Eurovision Song Contest. This involves predicting which 25 countries will make it to the final and then predicting the voting between these countries. The goal is to create a model that accurately predicts the voting results based on historical voting patterns, betting odds, song details, and artist details.\n\n2. The category of the problem is a classification and prediction problem. It involves predicting the categorical outcome of which countries will make it to the final and then predicting the numerical outcome of the voting scores between these countries.', metadata={'source': 'local'}),
 Document(page_content='this is

# Explaining similarity

In [37]:
prompt = PromptTemplate.from_template("""Here is a kaggle competition description {kaggle_desc}. Explain me how this competition is similar to this problem description {problem_desc}. Be concise""")
chat_model.predict(prompt.format(kaggle_desc=docs[0].page_content, problem_desc="Weather prediction"))

'Both the Kaggle competition and the weather prediction problem involve time series forecasting. In both cases, the goal is to predict future values based on historical data.'